In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

import gc

from os import path
import sys
sys.path.append(path.abspath('..'))

In [ ]:
import numpy as np
import onnx
import torch
import cv2
from PIL import Image
import tensorrt as trt
import matplotlib.pyplot as plt
from timm import create_model
import torch_tensorrt

from src.transforms import torch_preprocessing

In [ ]:
DEVICE = 'cuda:0'

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
# Загружаем и готовим торчовую модель
torch_model = create_model('gernet_l', pretrained=True)
_ = torch_model.to(DEVICE)
_ = torch_model.eval()

In [ ]:
# Загружаем картиночку
image = cv2.imread('../data/dog.jpg')[..., ::-1]
print(image.shape)
Image.fromarray(image)

## Батчевание

Есть 2 режима:
 - явный (explicit) - указываем размер батча
 - неявный (implicit) - не указываем размер батча
 
Но неявный "устарел" + не помню, чтобы в python действительно работал implicit размер батча. Поэтому всегда явный размер батча(

А если хочу разные размеры батча?! Есть решение, дальше увидим.

## Статический размер батча

In [ ]:
BATCH_SIZE = 1

In [ ]:
# готовим тензора для прогона
torch_input_tensor = torch_preprocessing(image).to(DEVICE)
torch_input_tensor = torch.cat([torch_input_tensor] * BATCH_SIZE)

### Torch

In [ ]:
# прогон через торчовую модель
with torch.no_grad():
    torch_output_tensor = torch_model(torch_input_tensor).cpu().detach().numpy()

In [ ]:
%%timeit
with torch.no_grad():
    torch_output_tensor = torch_model(torch_input_tensor).cpu().detach().numpy()

### TensorRT

In [ ]:
# готовим TensorRT модель с fp32
trt_model = torch_tensorrt.compile(
    torch_model,
    inputs = [torch_tensorrt.Input((BATCH_SIZE, 3, 224, 224))],
    enabled_precisions = torch.float32,
    workspace_size = 1 << 30, # 1 гибибайт
)

In [ ]:
# прогон через тенсоррт модель с fp32
with torch.no_grad():
    trt_output_tensor = trt_model(torch_input_tensor).cpu().detach().numpy()

In [ ]:
%%timeit
with torch.no_grad():
    trt_output_tensor = trt_model(torch_input_tensor).cpu().detach().numpy()

In [ ]:
# готовим TensorRT модель с fp16
trt_model_fp16 = torch_tensorrt.compile(
    torch_model,
    inputs = [torch_tensorrt.Input((BATCH_SIZE, 3, 224, 224))],
    enabled_precisions = torch.float16,
    workspace_size = 1 << 30, # 1 гибибайт
)

In [ ]:
# прогон через тенсоррт модель с fp16
with torch.no_grad():
    trt_output_tensor_fp16 = trt_model_fp16(torch_input_tensor).cpu().detach().numpy()

In [ ]:
%%timeit
with torch.no_grad():
    trt_output_tensor_fp16 = trt_model_fp16(torch_input_tensor).cpu().detach().numpy()

### Сравнение

In [ ]:
# Сравнение логитов
print(f'fp32: {list(np.abs(trt_output_tensor - torch_output_tensor).max(1))[0]}')
print(f'fp16: {list(np.abs(trt_output_tensor_fp16 - torch_output_tensor).max(1))[0]}')

In [ ]:
# Сравнение после активации
print(f'fp32: {list(np.abs(softmax(trt_output_tensor) - softmax(torch_output_tensor)).max(1))[0]}')
print(f'fp16: {list(np.abs(softmax(trt_output_tensor_fp16) - softmax(torch_output_tensor)).max(1))[0]}')

In [ ]:
# Финальный предикт
print(f'torch: {list(softmax(torch_output_tensor).argmax(1))[0]}')
print(f'fp32: {list(softmax(trt_output_tensor).argmax(1))[0]}')
print(f'fp16: {list(softmax(trt_output_tensor_fp16).argmax(1))[0]}')

## Динамический размер батча

In [ ]:
BATCH_SIZE = 2

In [ ]:
# готовим тензора для прогона
torch_input_tensor = torch_preprocessing(image).to(DEVICE)
torch_input_tensor = torch.cat([torch_input_tensor] * BATCH_SIZE)

### Torch

In [ ]:
# прогон через торчовую модель
with torch.no_grad():
    torch_output_tensor = torch_model(torch_input_tensor).cpu().detach().numpy()

In [ ]:
%%timeit
with torch.no_grad():
    torch_output_tensor = torch_model(torch_input_tensor).cpu().detach().numpy()

### TensorRT

In [ ]:
# готовим TensorRT модель с fp32
trt_model = torch_tensorrt.compile(
    torch_model,
    inputs = [
        torch_tensorrt.Input(
            min_shape=(1, 3, 224, 224),
            opt_shape=(3, 3, 224, 224),
            max_shape=(5, 3, 224, 224),
    )],
    enabled_precisions = torch.float32,
    workspace_size = 1 << 30, # 1 гибибайт
)

In [ ]:
# прогон через тенсоррт модель с fp32
with torch.no_grad():
    trt_output_tensor = trt_model(torch_input_tensor).cpu().detach().numpy()

In [ ]:
%%timeit
with torch.no_grad():
    trt_output_tensor = trt_model(torch_input_tensor).cpu().detach().numpy()

In [ ]:
# готовим TensorRT модель с fp16
trt_model_fp16 = torch_tensorrt.compile(
    torch_model,
    inputs = [
        torch_tensorrt.Input(
            min_shape=(1, 3, 224, 224),
            opt_shape=(3, 3, 224, 224),
            max_shape=(5, 3, 224, 224),
    )],
    enabled_precisions = torch.float16,
    workspace_size = 1 << 30, # 1 гибибайт
)

In [ ]:
# прогон через тенсоррт модель с fp16
with torch.no_grad():
    trt_output_tensor_fp16 = trt_model_fp16(torch_input_tensor).cpu().detach().numpy()

In [ ]:
%%timeit
with torch.no_grad():
    trt_output_tensor_fp16 = trt_model_fp16(torch_input_tensor).cpu().detach().numpy()

### Сравнение

In [ ]:
# Сравнение логитов
print(f'fp32: {list(np.abs(trt_output_tensor - torch_output_tensor).max(1))}')
print(f'fp16: {list(np.abs(trt_output_tensor_fp16 - torch_output_tensor).max(1))}')

In [ ]:
# Сравнение после активации
print(f'fp32: {list(np.abs(softmax(trt_output_tensor) - softmax(torch_output_tensor)).max(1))}')
print(f'fp16: {list(np.abs(softmax(trt_output_tensor_fp16) - softmax(torch_output_tensor)).max(1))}')

In [ ]:
# Финальный предикт
print(f'torch: {list(softmax(torch_output_tensor).argmax(1))}')
print(f'fp32: {list(softmax(trt_output_tensor).argmax(1))}')
print(f'fp16: {list(softmax(trt_output_tensor_fp16).argmax(1))}')

## Сохранение и загрузка чекпоинта

In [ ]:
# сохраняем на диск
torch.jit.save(trt_model, "../models/trt_torchscript_module.ts")

In [ ]:
# загружаем с диска
new_trt_model = torch.jit.load('../models/trt_torchscript_module.ts')

In [ ]:
with torch.no_grad():
    torch_output_tensor = new_trt_model(torch_input_tensor).cpu().detach().numpy()[0]
print(softmax(torch_output_tensor).argmax())

In [ ]:
%%timeit
with torch.no_grad():
    torch_output_tensor = new_trt_model(torch_input_tensor).cpu().detach().numpy()[0]